# 11. Map Bicycle Lanes GSV

Draw maps to compare the bicycle lane routes detected from GSV images in a survey area vs. the OpenStreetMap cycleways and the Principal Bicycle Network

## Configuration

Any configuration that is required to run this notebook can be customized in the next cell

In [4]:
# Which "locality" do we wish to process?
# Assumes that we can find a pair of OSM files with corresponding names,
# extracted with the "osmium" tool.  One file follows the official shape of
# the locality, while a second file follows a bounding box around the locality
# with a 200m margin, so that when we are looking for intersections, we don't
# miss any due to the intersecting road being just outside the boundary of the
# locality (apart from the intersection).
locality = 'Mount Eliza Sample'

# We will sample the middle of each intersection, but we can also sample a
# "margin" around the intersection, at 10m intervals.
# E.g. if we set this to "20" then we will sample points at:
#    -20m, -10m, 0m, 10m, and 20m
# from the centre of the intersection, along the assumed bearing of the road
margin = 20

# Default zoom level for maps
zoom = 14

## Code

In [5]:
# General imports
import json
import os
import sys

from ipyleaflet import Map, Marker, GeoJSON

import pandas as pd

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Import local modules
import osm_gsv_utils.detection_map as detection_map

In [6]:
# Derive paths for configuration

# A version of the locality name with spaces replaced by underscores
locality_clean   = locality.replace(' ', '_')

# Name of the locality with the margin around intersections included
locality_margin  = '{0:s}_{1:d}m'.format(locality_clean, margin)

# Detection log with individual points
detection_log = os.path.join(module_path_root, 'detections', locality_margin, 'detection_log.csv')

# Geojson files of detectsions and comparisons to OSM
detected_filename = os.path.join(module_path_root, 'detections', locality_margin, 'hit.geojson')
tag_filename      = os.path.join(module_path_root, 'detections', locality_margin, 'tag.geojson')
both_filename     = os.path.join(module_path_root, 'detections', locality_margin, 'both.geojson')
hit_only_filename = os.path.join(module_path_root, 'detections', locality_margin, 'hit_only.geojson')
tag_only_filename = os.path.join(module_path_root, 'detections', locality_margin, 'tag_only.geojson')

# Geojson file describing the outline of the survey area
margin_filename   = os.path.join(module_path_root, 'data_sources', 'Locality_{0:s}_margin.geojson'.format(locality_clean))

# ???
locality_filename = os.path.join(os.pardir, 'data_sources', 'LGA_boundary_Mount_Eliza.geojson')


In [7]:
# Load boundary of the sample area, so we can draw it on the map
margin_layer, margin_data = detection_map.load_layer(margin_filename)

center = detection_map.get_centroid(margin_data)

m_detect1 = Map(center=center, zoom=zoom)
m_detect1.add_layer(margin_layer)

# Load detected points and add to map
count = detection_map.load_points(m_detect1, detection_log)
print('Point count: ' + str(count))

# Display the map
m_detect1

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\Release\\minor_thesis\\detections\\Mount_Eliza_Sample_20m\\detection_log.csv'

## Display map of detected lines with boundary

In [ ]:
# Load boundary of the sample area, so we can draw it on the map
margin_layer, margin_data = detection_map.load_layer(margin_filename)

center = detection_map.get_centroid(margin_data)

# Load lines that we drew from the detected points
detected_layer, detected_data = detection_map.load_layer(detected_filename, color='red')

# Build the map
m_detect2 = Map(center=center, zoom=zoom)
m_detect2.add_layer(margin_layer)
m_detect2.add_layer(detected_layer)

# Display the map
m_detect2

## Display map of Open Street Map cycleways

In [ ]:
# Load boundary of the sample area, so we can draw it on the map
margin_layer, margin_data = detection_map.load_layer(margin_filename)

center = detection_map.get_centroid(margin_data)

# Load lines that we drew from the detected points
tag_layer, tag_data = detection_map.load_layer(tag_filename, color='red')

# Build the map
m_detect3 = Map(center=center, zoom=zoom)
m_detect3.add_layer(margin_layer)
m_detect3.add_layer(tag_layer)

# Display the map
m_detect3

## Comparison Map

In [ ]:
# Load boundary of the sample area, so we can draw it on the map
margin_layer, margin_data = detection_map.load_layer(margin_filename)

center = detection_map.get_centroid(margin_data)

# Load lines that we drew from the detected points
both_layer,     both_data     = detection_map.load_layer(both_filename,     color='green')
hit_only_layer, hit_only_data = detection_map.load_layer(hit_only_filename, color='blue')
tag_only_layer, tag_only_data = detection_map.load_layer(tag_only_filename, color='red')

# Build the map
m_detect4 = Map(center=center, zoom=zoom)
m_detect4.add_layer(margin_layer)
m_detect4.add_layer(both_layer)
m_detect4.add_layer(hit_only_layer)
m_detect4.add_layer(tag_only_layer)

# Display the map
m_detect4

OSM is missing a segment of Nepean Highway from Mount Eliza Way to Old Mornington Road.

This corresponds to way id="204762387".  It is NOT tagged with a "cycleway", it is tagged with "bicycle"="yes" and "foot"="yes".  This generally means that bicycle traffic (and foot traffic) is allowed, but not that there is a cycleway.

The small segment that is missing on Humphries Road from Walkers Road to Overport road was clipped in the OSM extract for the locality, it is just outside the official shape of the locality.  If we widen our OSM extract we can see that it is a cycleway.  way id="671976035".

In [ ]:
# locality_filename = os.path.join('data_sources', 'LGA_boundary_' + locality.replace(' ', '_') + '.geojson')
# pbn_existing_filename = os.path.join('data_sources', 'PBN_Existing.geojson')
# pbn_planned_filename = os.path.join('data_sources', 'PBN_Planned.geojson')
# osm_filename = os.path.join('data_sources', 'Locality_OSM_cycleway_' + locality.replace(' ', '_') + '.geojson')
# detect_filename = os.path.join('detections', locality_detect.replace(' ', '_'), 'detected_points.geojson')

## Demonstrate how to use Folium and save to .PNG

In [ ]:
import folium
import json

style_red = {
    'color': 'red',
    'weight': 2,
    'fillColor': 'red',
    'fillOpacity': 0.2
}

with open(margin_filename) as f:
    gj_margin = json.load(f)
    
with open(detected_filename) as f:
    gj_detected = json.load(f)

m = folium.Map(location=center, zoom_start=zoom)
folium.GeoJson(gj_margin, name='boundary').add_to(m)
folium.GeoJson(gj_detected, name='detected', style_function=lambda x:style_red).add_to(m)
m

In [ ]:
# Save folium to PNG, requires selenium, geckodriver, Firefox
import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save(os.path.join(os.pardir, 'map_detect.png'))

In [ ]:
import folium
import json

style_red = {
    'color': 'red',
    'weight': 2,
    'fillColor': 'red',
    'fillOpacity': 0.2
}

with open(margin_filename) as f:
    gj_margin = json.load(f)
    
with open(tag_filename) as f:
    gj_detected = json.load(f)

m = folium.Map(location=center, zoom_start=zoom)
folium.GeoJson(gj_margin, name='boundary').add_to(m)
folium.GeoJson(gj_detected, name='detected', style_function=lambda x:style_red).add_to(m)
m


In [ ]:
# Save folium to PNG, requires selenium, geckodriver, Firefox
import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save(os.path.join(os.pardir, 'map_osm.png'))

## Map of cycleways from wider local area

Use this to hunt for training footage outside the test area

In [ ]:
# Load boundary of the sample area, so we can draw it on the map
margin_layer, margin_data = detection_map.load_layer(margin_filename)

center = detection_map.get_centroid(margin_data)

# Load lines that we drew from the detected points
frankston_filename   = os.path.join(module_path_root, 'data_sources', 'Locality_OSM_cycleway_Frankston.geojson')
mornington_filename  = os.path.join(module_path_root, 'data_sources', 'Locality_OSM_cycleway_Mornington.geojson')
baxter_filename      = os.path.join(module_path_root, 'data_sources', 'Locality_OSM_cycleway_Baxter.geojson')
langwarrin_filename  = os.path.join(module_path_root, 'data_sources', 'Locality_OSM_cycleway_Langwarrin.geojson')

mount_eliza_filename = os.path.join(module_path_root, 'data_sources', 'Locality_OSM_cycleway_Mount_Eliza.geojson')


f_layer, f_data = detection_map.load_layer(frankston_filename,   color='red')
m_layer, m_data = detection_map.load_layer(mornington_filename,  color='red')
b_layer, b_data = detection_map.load_layer(baxter_filename,      color='green')
l_layer, l_data = detection_map.load_layer(langwarrin_filename,  color='orange')
e_layer, e_data = detection_map.load_layer(mount_eliza_filename, color='blue')

# Build the map
m_detect_c = Map(center=center, zoom=zoom)
m_detect_c.add_layer(margin_layer)
m_detect_c.add_layer(e_layer)
m_detect_c.add_layer(f_layer)
m_detect_c.add_layer(m_layer)
m_detect_c.add_layer(b_layer)
m_detect_c.add_layer(l_layer)

# Display the map
m_detect_c

## Load Dashcam Detections

In [ ]:
# Load boundary of the sample area, so we can draw it on the map
margin_layer, margin_data = detection_map.load_layer(locality_filename)

center = detection_map.get_centroid(margin_data)

# Load lines that we drew from the detected points
dashcam_dir = os.path.join(module_path_root, 'data_sources', 'dashcam_tour_mount_eliza', 'detections')

dashcam_hit = os.path.join(dashcam_dir, 'hit.geojson')
hit_layer, hit_data = detection_map.load_layer(dashcam_hit, color='blue')

# Build the map
m_detect = Map(center=center, zoom=zoom)
#m_detect.add_layer(margin_layer)
m_detect.add_layer(hit_layer)

# Display the map
m_detect

In [ ]:
lanes_filename = os.path.join(module_path_root, 'data_sources', 'dashcam_tour_mount_eliza', 'split', 'lanes.geojson')

# Load boundary of the sample area, so we can draw it on the map
margin_layer, margin_data = detection_map.load_layer(margin_filename)

center = detection_map.get_centroid(margin_data)

l_layer, l_data = detection_map.load_layer(lanes_filename, color='blue')

# Build the map
m_detect_l = Map(center=center, zoom=zoom)
#m_detect_l.add_layer(margin_layer)
m_detect_l.add_layer(l_layer)

# Display the map
m_detect_l
